[View in Colaboratory](https://colab.research.google.com/github/christopher11/machine-learning/blob/master/Neural_nets/Assignments/CHRISTOPHER_BATCH_4_ASSIGNMENT_4B.ipynb)

## Accuracy : 80.06 %

In [0]:
# https://keras.io/
!pip install -q keras
import keras

Using TensorFlow backend.


In [0]:
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam

In [0]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [0]:
# Hyperparameters
batch_size = 256
num_classes = 10
epochs = 50
l = 12
num_filter = 18
compression = 0.6
dropout_rate = 0.2

In [0]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

170500096/170498071 [==============================] - 32s 0us/step


In [0]:
# Dense Block
def add_denseblock(input, num_filter = 32, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [0]:
def add_transition(input, num_filter = 32, dropout_rate = 0.2):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg

In [0]:
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output

In [0]:
num_filter = 12
dropout_rate = 0.2
l =12
input = Input(shape=(img_height, img_width, channel,))
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)


In [0]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_241 (Conv2D)             (None, 32, 32, 12)   324         input_5[0][0]                    
__________________________________________________________________________________________________
batch_normalization_241 (BatchN (None, 32, 32, 12)   48          conv2d_241[0][0]                 
__________________________________________________________________________________________________
activation_241 (Activation)     (None, 32, 32, 12)   0           batch_normalization_241[0][0]    
__________________________________________________________________________________________________
conv2d_242

In [0]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

In [0]:
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/50
50000/50000 [==============================] - 157s 3ms/step - loss: 1.7543 - acc: 0.3451 - val_loss: 2.4378 - val_acc: 0.2849
Epoch 2/50
50000/50000 [==============================] - 135s 3ms/step - loss: 1.4047 - acc: 0.4793 - val_loss: 1.6993 - val_acc: 0.3964
Epoch 3/50
 9216/50000 [====>.........................] - ETA: 1:43 - loss: 1.2999 - acc: 0.5204

50000/50000 [==============================] - 135s 3ms/step - loss: 1.2597 - acc: 0.5364 - val_loss: 1.2204 - val_acc: 0.5646
Epoch 4/50
50000/50000 [==============================] - 135s 3ms/step - loss: 1.1531 - acc: 0.5793 - val_loss: 1.3178 - val_acc: 0.5565
Epoch 5/50
29952/50000 [================>.............] - ETA: 50s - loss: 1.0809 - acc: 0.6113

50000/50000 [==============================] - 134s 3ms/step - loss: 1.0698 - acc: 0.6148 - val_loss: 1.2935 - val_acc: 0.5844
Epoch 6/50
50000/50000 [==============================] - 134s 3ms/step - loss: 1.0136 - acc: 0.6319 - val_loss: 1.0223 - val_acc: 0.6415
Epoch 7/50
34816/50000 [===================>..........] - ETA: 37s - loss: 0.9606 - acc: 0.6550

50000/50000 [==============================] - 134s 3ms/step - loss: 0.9597 - acc: 0.6547 - val_loss: 1.1130 - val_acc: 0.6274
Epoch 8/50
50000/50000 [==============================] - 134s 3ms/step - loss: 0.9159 - acc: 0.6704 - val_loss: 1.0883 - val_acc: 0.6399
Epoch 9/50
35840/50000 [====================>.........] - ETA: 35s - loss: 0.8873 - acc: 0.6802

50000/50000 [==============================] - 134s 3ms/step - loss: 0.8836 - acc: 0.6816 - val_loss: 1.0762 - val_acc: 0.6466
Epoch 10/50
50000/50000 [==============================] - 134s 3ms/step - loss: 0.8572 - acc: 0.6903 - val_loss: 1.0218 - val_acc: 0.6568
Epoch 11/50
35840/50000 [====================>.........] - ETA: 35s - loss: 0.8289 - acc: 0.7026

...

50000/50000 [==============================] - 133s 3ms/step - loss: 0.4908 - acc: 0.8252 - val_loss: 0.6522 - val_acc: 0.7959
Epoch 42/50
50000/50000 [==============================] - 134s 3ms/step - loss: 0.4909 - acc: 0.8258 - val_loss: 0.6381 - val_acc: 0.8028
Epoch 43/50
35840/50000 [====================>.........] - ETA: 35s - loss: 0.4771 - acc: 0.8301

50000/50000 [==============================] - 133s 3ms/step - loss: 0.4802 - acc: 0.8302 - val_loss: 0.6192 - val_acc: 0.8012
Epoch 44/50
50000/50000 [==============================] - 134s 3ms/step - loss: 0.4735 - acc: 0.8294 - val_loss: 0.6850 - val_acc: 0.7882
Epoch 45/50
35840/50000 [====================>.........] - ETA: 35s - loss: 0.4655 - acc: 0.8328

50000/50000 [==============================] - 134s 3ms/step - loss: 0.4691 - acc: 0.8315 - val_loss: 0.8951 - val_acc: 0.7444
Epoch 46/50
50000/50000 [==============================] - 134s 3ms/step - loss: 0.4690 - acc: 0.8330 - val_loss: 0.8214 - val_acc: 0.7581
Epoch 47/50
35840/50000 [====================>.........] - ETA: 35s - loss: 0.4647 - acc: 0.8362

50000/50000 [==============================] - 134s 3ms/step - loss: 0.4678 - acc: 0.8354 - val_loss: 0.7013 - val_acc: 0.7847
Epoch 48/50
50000/50000 [==============================] - 134s 3ms/step - loss: 0.4595 - acc: 0.8365 - val_loss: 0.6516 - val_acc: 0.7949
Epoch 49/50
35840/50000 [====================>.........] - ETA: 35s - loss: 0.4538 - acc: 0.8390

50000/50000 [==============================] - 134s 3ms/step - loss: 0.4578 - acc: 0.8386 - val_loss: 0.7819 - val_acc: 0.7725
Epoch 50/50
50000/50000 [==============================] - 136s 3ms/step - loss: 0.4539 - acc: 0.8386 - val_loss: 0.6593 - val_acc: 0.8006


In [0]:
# Test the model
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 15s 1ms/step
Test loss: 0.6592810513496399
Test accuracy: 0.8006


In [0]:
# Save the trained weights in to .h5 format
model.save_weights("DNST_model.h5")
print("Saved model to disk")

Saved model to disk


In [0]:
from google.colab import files

files.download('DNST_model.h5')